<a href="https://colab.research.google.com/github/JasonD28/CS-175/blob/master/model/faster_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pydicom

In [3]:
%%shell
git clone https://github.com/JasonD28/CS-175.git
cd CS-175
cp model/dataset.py  ../

fatal: destination path 'CS-175' already exists and is not an empty directory.


In [4]:
%%shell
# Using PyTorch helper functions to simplify training
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.5.1

cp references/detection/utils.py ../
cp references/detection/engine.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/coco_utils.py ../

fatal: destination path 'vision' already exists and is not an empty directory.
Already on 'v0.5.1'
Your branch is up to date with 'origin/v0.5.1'.


In [0]:
import torch
from torch.autograd import Variable
import utils
from torch import optim
import torchvision

from dataset import PneumoniaDataset
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

root = "drive/My Drive/cs-175-project/cs-175-data"

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=3)
model.load_state_dict(torch.load('drive/My Drive/cs-175-project/model_checkpoints/eighth_try_model.pt'))

model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequent

In [0]:
testset = PneumoniaDataset(root, True, 'test')
test_loader = torch.utils.data.DataLoader(
        testset, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

In [0]:
import pandas as pd
final = pd.DataFrame(columns = ['patientId', 'PredictionString'])
confidence = 0.5

In [9]:
model.eval()
gpu_dtype = torch.cuda.FloatTensor
predictions = []
for i, (x, y) in enumerate(test_loader):
  with torch.no_grad():
    images = list(image.to(device) for image in x)
  pred = model(images)[0]
  preds = pred
  pred_str = []
  for index, score in enumerate(preds['scores'].tolist()):
    if score >= confidence:
      boxes = preds['boxes'][index].tolist()
      if preds['labels'][index] == 2:
        pred_str.extend([score, boxes[0], boxes[1], boxes[2] - boxes[0], boxes[3] - boxes[1]])
  pred_str = [str(x) for x in pred_str]
  pred_str = ' '.join(pred_str)
  final = final.append({'patientId': testset.imgs[i][:-4], 'PredictionString': pred_str}, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


In [0]:
# import pandas as pd
# final = pd.DataFrame(columns = ['patientId', 'PredictionString'])

In [0]:
# confidence = 0.5
# for img_id, preds in predictions:
#   preds = preds[0]
#   pred_str = []
#   for index, score in enumerate(preds['scores'].tolist()):
#     if score >= confidence:
#       boxes = preds['boxes'][index].tolist()
#       # if preds['labels'][index] == 2:
#         pred_str.extend([score, boxes[0], boxes[1], boxes[2] - boxes[0], boxes[3] - boxes[1]])
#   pred_str = [str(x) for x in pred_str]
#   pred_str = ' '.join(pred_str)
#   final = final.append({'patientId': img_id, 'PredictionString': pred_str}, ignore_index=True)

In [12]:
print(final)

                                 patientId                                   PredictionString
0     0000a175-0e68-4ca4-b1af-167204a7e0bc                                                   
1     0005d3cc-3c3f-40b9-93c3-46231c3eb813                                                   
2     000686d7-f4fc-448d-97a0-44fa9c5d3aa6                                                   
3     000e3a7d-c0ca-4349-bb26-5af2d8993c3d                                                   
4     00100a24-854d-423d-a092-edcf6179e061  0.5938289165496826 529.087158203125 362.260467...
...                                    ...                                                ...
2995  c1e88810-9e4e-4f39-9306-8d314bfc1ff1                                                   
2996  c1ec035b-377b-416c-a281-f868b7c9b6c3                                                   
2997  c1ef5b66-0fd7-49d1-ae6b-5af84929414b                                                   
2998  c1ef6724-f95f-40f1-b25b-de806d9bc39d                  

In [0]:
final.to_csv('drive/My Drive/cs-175-project/predictions/faster_prediction.csv', index=False)